# Double bottleneck simulation 

In this demonstration, we'll construct a **double bottleneck** situation and simulate the evacuation of **10 agents** positioned on a grid.

**TODO: MORE EXPLANATION. What happens here? Why? Why do we hope to show?**


***Introduction***[CB]

The simulation of the double bottleneck is based on the work for the (single) bottleneck. 
Here, agents are positioned on both side of the bottleneck: left and right.

How will the agents "meet"? 
Will they disturb each other? Is it even possible to go through? 
And what is the impact of the width of the bottleneck?

In order to answer those questions, please follow the instructions below. 

Let's begin by importing the required packages for our simulation:

In [ ]:
from shapely import GeometryCollection, Polygon, to_wkt
import pathlib
import pandas as pd
import numpy as np
import jupedsim as jps
from jupedsim.distributions import distribute_by_number
import sqlite3  # parse trajectory db
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Figure
import pedpy  # analysis

%matplotlib inline

## Setting up a geometry

In [ ]:
p1 = Polygon([(0, 0), (10, 0), (10, 10), (0, 10)])
p2 = Polygon([(10, 4), (15, 4), (15, 6), (10, 6)])
p3 = Polygon([(15, 0), (25, 0), (25, 10), (15, 10)])
area = GeometryCollection(p1.union(p2.union(p3)))
walkable_area = pedpy.WalkableArea(area.geoms[0])
pedpy.plot_walkable_area(walkable_area=walkable_area)

## Operational model
<a id="model"></a>
Once the geometry is set, our subsequent task is to specify the model and its associated parameters.
For this demonstration, we'll employ the "collision-free" model.

## Setting Up the Simulation Object

Having established the model and geometry details, and combined with other parameters such as the time step dt, we can proceed to construct our simulation object as illustrated below:

In [ ]:
trajectory_file = "double_bottleneck.sqlite"
simulation = jps.Simulation(
    model=jps.VelocityModel(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

## Specifying Routing Details

At this juncture, we'll provide basic routing instructions, guiding the agents to progress towards the **first exit point** and the **second exit point**.

In [ ]:
exits = [
    simulation.add_exit_stage([(24, 0), (25, 0), (25, 10), (24, 10)]),
    simulation.add_exit_stage([(0, 0), (1, 0), (1, 10), (0, 10)]),
]
journeys = [
    simulation.add_journey(jps.JourneyDescription([exit])) for exit in exits
]

## Defining and Distributing Agents

Now, we'll position the agents and establish their attributes, leveraging previously mentioned parameters such as `exit1`, `exit2` and `profile_id`.

In [ ]:
agent_parameters = jps.VelocityModelAgentParameters()
agent_parameters.journey_id = journeys[0]
agent_parameters.stage_id = exits[0]
agent_parameters.orientation = (1.0, 0.0)
agent_parameters.v0 = 1.2
agent_parameters.radius = 0.15
agent_parameters.time_gap = 1

for position in [(7, 7), (1, 3), (1, 5), (1, 7), (2, 7)]:
    simulation.add_agent(
        jps.VelocityModelAgentParameters(
            journey_id=journeys[0], stage_id=exits[0], position=position
        )
    )
for position in [(25, 7), (21, 3), (21, 5), (21, 7), (22, 7)]:
    simulation.add_agent(
        jps.VelocityModelAgentParameters(
            journey_id=journeys[1], stage_id=exits[1], position=position
        )
    )

## Executing the Simulation

With all components in place, we're set to initiate the simulation.
For this demonstration, the trajectories will be recorded in an sqlite database.

In [ ]:
while simulation.agent_count() > 0:
    simulation.iterate()

## Visualizing the Trajectories

For trajectory visualization, we'll extract data from the sqlite database. A straightforward method for this is employing the jupedsim-visualizer.

To-Do List:

    Incorporate references and hyperlinks to additional resources.
    Integrate results visualization using the visualizer.

In [ ]:
def read_sqlite_file(
    trajectory_file: str,
) -> (pedpy.TrajectoryData, pedpy.WalkableArea):
    with sqlite3.connect(trajectory_file) as con:
        data = pd.read_sql_query(
            "select frame, id, pos_x as x, pos_y as y, ori_x as ox, ori_y as oy from trajectory_data",
            con,
        )
        fps = float(
            con.cursor()
            .execute("select value from metadata where key = 'fps'")
            .fetchone()[0]
        )
        walkable_area = (
            con.cursor().execute("select wkt from geometry").fetchone()[0]
        )
        return (
            pedpy.TrajectoryData(data=data, frame_rate=fps),
            pedpy.WalkableArea(walkable_area),
        )


def speed_to_color(speed, min_speed, max_speed, midpoint):
    colorscale = px.colors.diverging.RdBu_r[::-1]

    # Normalize speed based on the midpoint
    if speed >= midpoint:
        normalized_speed = 0.5 + 0.5 * (speed - midpoint) / (
            max_speed - midpoint
        )
    else:
        normalized_speed = 0.5 * (speed - min_speed) / (midpoint - min_speed)

    # Clip to ensure the value is between 0 and 1
    normalized_speed = np.clip(normalized_speed, 0, 1)

    # Find the corresponding color in the colorscale
    color_idx = int(normalized_speed * (len(colorscale) - 1))
    return colorscale[color_idx]


def get_geometry_traces(area):
    geometry_traces = []
    x, y = area.exterior.xy
    geometry_traces.append(
        go.Scatter(
            x=np.array(x),
            y=np.array(y),
            mode="lines",
            line={"color": "grey"},
            showlegend=False,
            name="Exterior",
            hoverinfo="name",
        )
    )
    for inner in area.interiors:
        xi, yi = zip(*inner.coords[:])
        geometry_traces.append(
            go.Scatter(
                x=np.array(xi),
                y=np.array(yi),
                mode="lines",
                line={"color": "grey"},
                showlegend=False,
                name="Obstacle",
                hoverinfo="name",
            )
        )
    return geometry_traces


def get_shapes_for_frame(frame_data, min_speed, max_speed, midpoint):
    def create_shape(row):
        hover_trace = go.Scatter(
            x=[row["x"]],
            y=[row["y"]],
            text=[f"ID: {row['id']}, Pos({row['x']:.2f},{row['y']:.2f})"],
            mode="markers",
            marker=dict(size=1, opacity=1),
            hoverinfo="text",
            showlegend=False,
        )
        if row["speed"] == -1000:  # Check for dummy speed
            return (
                go.layout.Shape(
                    type="circle",
                    xref="x",
                    yref="y",
                    x0=row["x"] - row["radius"],
                    y0=row["y"] - row["radius"],
                    x1=row["x"] + row["radius"],
                    y1=row["y"] + row["radius"],
                    line=dict(width=0),
                    fillcolor="rgba(255,255,255,0)",  # Transparent fill
                ),
                hover_trace,
            )
        color = speed_to_color(row["speed"], min_speed, max_speed, midpoint)
        return (
            go.layout.Shape(
                type="circle",
                xref="x",
                yref="y",
                x0=row["x"] - row["radius"],
                y0=row["y"] - row["radius"],
                x1=row["x"] + row["radius"],
                y1=row["y"] + row["radius"],
                line_color=color,
                fillcolor=color,
            ),
            hover_trace,
        )

    results = frame_data.apply(create_shape, axis=1).tolist()
    shapes = [res[0] for res in results]
    hover_traces = [res[1] for res in results]
    return shapes, hover_traces


def create_fig(
    initial_agent_count,
    initial_shapes,
    initial_hover_trace,
    geometry_traces,
    hover_traces,
    frames,
    steps,
    area_bounds,
    width=800,
    height=800,
):
    minx, miny, maxx, maxy = area_bounds
    fig = go.Figure(
        data=geometry_traces + hover_traces + initial_hover_trace,
        frames=frames,
        layout=go.Layout(
            shapes=initial_shapes,
            title=f"<b>Number of Agents: {initial_agent_count}</b>",
            title_x=0.5,
        ),
    )
    fig.update_layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [
                            None,
                            {
                                "frame": {"duration": 100, "redraw": True},
                                "fromcurrent": True,
                            },
                        ],
                        "label": "Play",
                        "method": "animate",
                    }
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top",
            }
        ],
        sliders=[
            {
                "active": 0,
                "yanchor": "top",
                "xanchor": "left",
                "currentvalue": {
                    "font": {"size": 20},
                    "prefix": "Frame:",
                    "visible": True,
                    "xanchor": "right",
                },
                "transition": {"duration": 100, "easing": "cubic-in-out"},
                "pad": {"b": 10, "t": 50},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": steps,
            }
        ],
        autosize=False,
        width=width,
        height=height,
        xaxis=dict(range=[minx - 0.5, maxx + 0.5]),
        yaxis=dict(
            scaleanchor="x", scaleratio=1, range=[miny - 0.5, maxy + 0.5]
        ),
    )
    return fig


def animate(
    data: pedpy.TrajectoryData, area: pedpy.WalkableArea, *, every_nth_frame=5
):
    data_df = pedpy.compute_individual_speed(traj_data=data, frame_step=5)
    data_df = data_df.merge(data.data, on=["id", "frame"], how="left")
    data_df["radius"] = 0.2
    min_speed = data_df["speed"].min()
    max_speed = data_df["speed"].max()
    midpoint = np.mean(data_df["speed"])
    max_agents = data_df.groupby("frame").size().max()
    dummy_agent_data = {"x": 0, "y": 0, "radius": 0, "speed": -1000}
    frames = []
    steps = []
    unique_frames = data_df["frame"].unique()
    selected_frames = unique_frames[::every_nth_frame]
    geometry_traces = get_geometry_traces(area.polygon)
    initial_frame_data = data_df[data_df["frame"] == data_df["frame"].min()]
    initial_agent_count = len(initial_frame_data)
    initial_shapes, initial_hovers = get_shapes_for_frame(
        initial_frame_data, min_speed, max_speed, midpoint
    )
    for frame_num in selected_frames[1:]:
        frame_data = data_df[data_df["frame"] == frame_num]
        agent_count = len(frame_data)
        while len(frame_data) < max_agents:
            dummy_df = pd.DataFrame([dummy_agent_data])
            frame_data = pd.concat([frame_data, dummy_df], ignore_index=True)

        shapes, hover_traces = get_shapes_for_frame(
            frame_data, min_speed, max_speed, midpoint
        )
        frame = go.Frame(
            data=geometry_traces + hover_traces,
            name=str(frame_num),
            layout=go.Layout(
                shapes=shapes,
                title=f"<b>Number of Agents: {agent_count}</b>",
                title_x=0.5,
            ),
        )
        frames.append(frame)
        step = {
            "args": [
                [str(frame_num)],
                {
                    "frame": {"duration": 100, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 500},
                },
            ],
            "label": str(frame_num),
            "method": "animate",
        }
        steps.append(step)

    return create_fig(
        initial_agent_count,
        initial_shapes,
        initial_hovers,
        geometry_traces,
        hover_traces,
        frames,
        steps,
        area.bounds,
        width=800,
        height=800,
    )

In [ ]:
trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area)

## References & Further Exploration

The operational model discussed in the Model section is based on the collision-free model. JuPedSim also incorporates another model known as GCFM. For more details on GCFM, refer to another notebook (TODO: Link to the GCFM notebook).

Our demonstration employed a straightforward journey with a singular exit. For a more intricate journey featuring multiple intermediate stops and waiting zones, see the upcoming section (TODO: Link to the advanced journey section).

While we designated a single parameter profile for agents in this example, it's feasible to define multiple parameter profiles. Learn how to alternate between these profiles in the subsequent section (TODO: Link to the profile-switching section).